# Лабораторная работа 3 : Деревья решений. Ансамбли решающих деревьев

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import plot_tree
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import xgboost as xgb


## Задание 1

In [ ]:
# V = 83 => вариант 6

df = pd.read_csv('train.csv')
df['Cover_Type'] -= 1
df


In [ ]:
res = {
    0: 'Spruce/Fir',
    1: 'Lodgepole Pine',
    2: 'Ponderosa Pine',
    3: 'Cottonwood/Willow',
    4: 'Aspen',
    5: 'Douglas-fir',
    6: 'Krummholz'
}


In [ ]:
print('статистики')
df.describe()


In [ ]:
print('пропущенных значений', df.apply(lambda x: sum(x.isnull()), axis=0).sum())


In [ ]:
X = df.dropna().reset_index().drop('Cover_Type', axis=1)
y = df.dropna().reset_index()['Cover_Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2022)


## Задание 2

In [ ]:
decision_tree = DecisionTreeClassifier(max_depth=3)
decision_tree.fit(X_train, y_train)
y_pred = decision_tree.predict(X_test)


In [ ]:
plt.figure(figsize=(25, 25))
plot_tree(decision_tree, feature_names=df.drop("Cover_Type", axis=1).columns, fontsize=9, class_names=list(res.values()), filled=True)


Если Aspect не больше 2377.5 и Hillshade_Noon не больше 194.5, то это Ponderosa Pine.

Если Aspect не больше 2377.5 и Hillshade_Noon больше 194.5, то это Cottonwood/Willow.

Если Aspect больше 2377.5, но меньше 2672.5 и Soil_Type11 не больше 0.5, то это Ponderosa Pine.

Если Aspect больше 2377.5, но меньше 2672.5 и Soil_Type11 больше 0.5, то это Douglas-fir.

Если Aspect больше 2672.5, но не больше 2959.5, то это Aspen.

Если Aspect больше 2959.5, но не больше 3232.5, то это Spruce/Fir.

Если Aspect больше 3232.5, то это Krummholz.

In [ ]:
print('точность', accuracy_score(y_pred, y_test))


In [ ]:
print('f1 score', f1_score(y_pred, y_test, average='micro'))


In [ ]:
y_score = decision_tree.predict_proba(X_test)
print('roc auc score', roc_auc_score(y_test, y_score, multi_class='ovr'))


## Часть 3

In [ ]:
# “gini”, “entropy”, “log_loss”
crit = ['gini', 'entropy']
md = 15

scores = np.zeros((2, md))

for i in range(2):
    for n_nodes in range(1, md + 1):
        clf = DecisionTreeClassifier(max_depth=n_nodes, criterion=crit[i])
        score = cross_val_score(clf, X, y, scoring='f1_macro')
        scores[i][n_nodes - 1] = np.mean(score)

plt.subplots(figsize=[10, 10])
plt.plot([i for i in range(1, md + 1)], scores[0], label=crit[0])
plt.plot([i for i in range(1, md + 1)], scores[1], label=crit[1])
plt.legend()


Видно, что результаты слабо зависят от выбора критерия. Они показывают схожие результаты. Максимальная точность достигается при глубине от 4 до 8, далее идет снижение качества.

## Задание 4

In [ ]:
#clf = GradientBoostingClassifier()
clf = xgb.XGBClassifier()
clf.fit(X_train, y_train)


In [ ]:
print('f1 score (train)', f1_score(y_train, clf.predict(X_train), average='micro'))
print('f1 score (test)', f1_score(y_test, clf.predict(X_test), average='micro'))


In [ ]:
# from pydotplus import graph_from_dot_data
# #from IPython.display import Image
# for i in range(3):
#     dot_data = export_graphviz(
#         clf.estimators_[i, 0],
#         out_file=None, filled=True, rounded=True,
#         special_characters=True,
#         proportion=False, impurity=False,
#     )
#     plt.figure(figsize=(12,12))
#     graph = graph_from_dot_data(dot_data)
#     Image(graph.create_png())
#     plt.show()

for i in range(3):
    fig, ax = plt.subplots(figsize=(30, 30))
    xgb.plot_tree(clf, num_trees=i, ax=ax, rankdir='LR')
    plt.show()


In [ ]:
imp_dict = dict()
feature_imp = clf.feature_importances_

for feature, imp in zip(X.columns, feature_imp):
    if imp != 0:
        imp_dict[feature] = imp

sorted_imp = sorted(imp_dict.items(), key=lambda kv: kv[1], reverse=True)

print('feature importances')

for num, (feature, imp) in enumerate(sorted_imp):
    print(feature, imp)


In [ ]:
md = 10

scores = np.zeros(md)

for n_nodes in range(1, md + 1):
    clf = GradientBoostingClassifier(n_estimators=n_nodes, max_depth=7, learning_rate=0.2)
    score = cross_val_score(clf, X, y, scoring='f1_macro')
    scores[n_nodes - 1] = np.mean(score)

plt.subplots(figsize=[10, 10])
plt.plot([i for i in range(1, md + 1)], scores, label='score')
plt.legend()


Видно, что для градиентного бустинга оптимальным является 4 дерева, после чего идет ухудшение качества.

## Часть 5

In [ ]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)


In [ ]:
print('f1 score (train)', f1_score(y_train, clf.predict(X_train), average='micro'))
print('f1 score (test)', f1_score(y_test, clf.predict(X_test), average='micro'))


In [ ]:
imp_dict = dict()
feature_imp = clf.feature_importances_

for feature, imp in zip(X.columns, feature_imp):
    if imp != 0:
        imp_dict[feature] = imp

sorted_imp = sorted(imp_dict.items(), key=lambda kv: kv[1], reverse=True)

print('feature importances')

for num, (feature, imp) in enumerate(sorted_imp):
    print(feature, imp)


In [ ]:
md = 20

scores = np.zeros(md)

for n_nodes in range(1, md + 1):
    clf = RandomForestClassifier(n_estimators=n_nodes)
    score = cross_val_score(clf, X, y, scoring='f1_macro')
    scores[n_nodes - 1] = np.mean(score)

plt.subplots(figsize=[10, 10])
plt.plot([i for i in range(1, md + 1)], scores, label='score')
plt.legend()


Видно, что для случайного леса не происходит снижения качества при увеличении количества деревьев.